In [1]:
using DataFrames, NPZ, Arrow, JLD2
using Statistics, NaNMath

In [2]:
Df = jldopen("data/timelapses/profile_database.jld2", "r")["df"];


┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 /home/pablo/.julia/packages/JLD2/VHRWL/src/JLD2.jl:230


LoadError: SystemError: opening file "data/timelapses/profile_database.jld2": No such file or directory